# **Tutoriel : le paradigme columnar avec ClickHouse**

### **Introduction**

Bienvenue dans notre tutoriel sur le moteur de base de données Clickhouse, basé sur le paradigme columnar. Ce tutoriel permet de présenter le paradigme columnar, d'étudier ses spécificités, ses atouts et ses limites. A cette fin, il vous est proposé de travailler sur un ensemble de données des taxis de la ville de New York. Vous allez manipuler ces données à la fois avec un moteur classique en MySQL et avec un moteur en columnar grâce à ClickHouse, avec pour objectif de faire la comparaison entre les deux approches pour mettre en valeur les singularités du paradigme columnar.

Dans la majorité des bases de données relationelles, les données sont structurées et conservées sous forme de lignes. Pour chaque ligne il y a une clé primaire à laquelle sont associées des propriétés. Dans le cas du paradigme columnar, les données sont conservées sous forme de colonnes et non de lignes.
Pour illustrer cette distinction entre les deux approches, ci-dessous un exemple tiré de notre jeu de données.


| ID    | Passenger Count  | Trip Distance  | Total amount  |
| ----- | ---------------- | -------------- | ------------- |
| 0     | 1                | 18             | 50            |
| 1     | 2                | 3              | 8             |
| 2     | 1                | 7              | 17            |
| 3     | 4                | 10             | 32            |

Dans une base de données orientée lignes, les données ci-dessus sont stockées selon les lignes :

(0, 1, 18, 50) ; (1, 2, 3, 8) ; (2, 1, 7, 17) ; (3, 4, 10, 32).

Tandis que dans une base de données columnar, la rétention se fait selon les colonnes :

(0, 1, 2, 3) ; (1, 2, 1, 4) ; (18, 3, 7, 10) ; (50, 8, 17, 32).

### **Mise en place**

Pour commencer, il faut créer et démarrer les conteneurs encapsulant les différents composants nécessaires pour ce tutoriel, avec une commande `docker compose up`.

Ensuite il faut se connecter au serveur Jupyter accessible à l'adresse http://localhost:8888.

Enfin il faut configurer les connecteurs pour interagir avec les moteurs MySQL et ClickHouse. `conn` est le connecteur MySQL tandis que `client` est le client ClickHouse.

In [1]:
import pandas as pd
import time
from utils import test_mysql, test_clickhouse, sql_query, conn, client

### **Vérification des conteneurs**

Exécutez les blocs ci-dessous pour valider la bonne connection avec les conteneurs de moteurs de base de données.

In [2]:
# Le résultat attendu est "MySQL : (1, 'test', 100)"
test_mysql()

MySQL : (1, 'test', 100)


In [3]:
# Le résultat attendu est "ClickHouse : (1, 'test', 100)"
test_clickhouse()

ClickHouse : (1, 'test', 100)


Chargement du jeu de données avec les informations sur des trajets de taxi à New York.
Le jeu de données a été réduit à 50 000 données.

In [4]:
df = pd.read_csv(
    "./dataset/input_data.csv",
    usecols=[
        "VendorID",
        "passenger_count",
        "trip_distance",
        "fare_amount",
        "tip_amount",
        "tolls_amount",
    ],
)

## Découverte de la syntaxe

`client.command(query)` permet d'effectuer les opérations sur les tables (création, destruction).

In [5]:
client.command("DROP TABLE IF EXISTS dnd;")
client.command("""
    CREATE TABLE dnd (
        id UInt32,
        name VARCHAR(24),
        age UInt8,
        strengh Float32,
        charisma Float32,
        agility Float32,
        intelligence Float32
    ) ENGINE = MergeTree()
    ORDER BY id;
""")

`client.insert(table, data)` permet d'ajouter un ensemble de données à une table, où `data` prend la forme d'une liste de tuples représentant un tableau.

`data = [(...), (...), ...]`

In [6]:
data = [
    (1, "Alice", 23, 60, 80, 50, 55),
    (2, "Bob", 24, 60, 75, 65, 45),
    (3, "Charlie", 23, 90, 60, 45, 50),
    (4, "David", 23, 70, 70, 55, 50),
    (5, "Eleanore", 22, 60, 80, 40, 65),
]
client.insert("dnd", data)

`client.query(query)` permet d'effectuer une requête pour obtenir des données

In [7]:
result = client.query("SELECT * FROM dnd")

## Comparaison 1 : Chargement des données

In [8]:
NB_ROWS_TO_INSERT = 5000

On charge `NB_ROWS_TO_INSERT` données une par une pour étudier le temps mis par chaque système pour les ajouter.

In [9]:
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS nyc_taxi;")

cursor.execute("""
    CREATE TABLE nyc_taxi (
        id INT AUTO_INCREMENT PRIMARY KEY,
        vendor_id INT,
        passenger_count INT,
        trip_distance FLOAT,
        fare_amount FLOAT,
        tip_amount FLOAT,
        tolls_amount FLOAT
    );
""")

conn.commit()

t0 = time.time()
for i, row in df.iterrows():
    cursor.execute("""
        INSERT INTO nyc_taxi (vendor_id, passenger_count, trip_distance, fare_amount, tip_amount, tolls_amount)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, tuple(row))
    conn.commit()
    if i == NB_ROWS_TO_INSERT :
        break
t1 = time.time()
print("Time required to add %d rows one by one : "%(NB_ROWS_TO_INSERT), t1 - t0)

cursor.close()

Time required to add 5000 rows one by one :  12.723230838775635


True

In [10]:
client.command("DROP TABLE IF EXISTS nyc_taxi;")

client.command("""
    CREATE TABLE nyc_taxi (
        id UInt32,
        vendor_id UInt8,
        passenger_count UInt8,
        trip_distance Float32,
        fare_amount Float32,
        tip_amount Float32,
        tolls_amount Float32
    ) ENGINE = MergeTree()
    ORDER BY id;
""")

t0 = time.time()
data = []
for i, row in df.iterrows():
    client.insert("nyc_taxi", [(i, row["VendorID"], row["passenger_count"], row["trip_distance"], row["fare_amount"], row["tip_amount"], row["tolls_amount"])])
    if i == NB_ROWS_TO_INSERT :
        break
t1 = time.time()
print("Time required to add %d rows one by one : "%(NB_ROWS_TO_INSERT), t1 - t0)

Time required to add 5000 rows one by one :  22.41345763206482


On observe que ClickHouse est beaucoup plus lent que MySQL pour l'insertion des données.
Cela est dû à la différence dans la façon de conserver les données.
- Pour MySQL, on ajoute 1 liste de N éléments
- Pour ClickHouse, on ajoute 1 élement dans N listes

Afin de pallier ce problème, on utilise la fonction `insert` avec un tableau de données pour ajouter un paquet plutôt que des données une par une.

Ajoutez à présent l'ensemble des données dans les deux bases de données (cela devrait prendre quelques minutes).

In [11]:
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS nyc_taxi;")

cursor.execute("""
    CREATE TABLE nyc_taxi (
        id INT AUTO_INCREMENT PRIMARY KEY,
        vendor_id INT,
        passenger_count INT,
        trip_distance FLOAT,
        fare_amount FLOAT,
        tip_amount FLOAT,
        tolls_amount FLOAT
    );
""")

conn.commit()

for i, row in df.iterrows():
    cursor.execute("""
        INSERT INTO nyc_taxi (vendor_id, passenger_count, trip_distance, fare_amount, tip_amount, tolls_amount)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, tuple(row))

conn.commit()
cursor.close()

True

In [12]:
client.command("DROP TABLE IF EXISTS nyc_taxi;")

client.command("""
    CREATE TABLE nyc_taxi (
        id UInt32,
        vendor_id UInt8,
        passenger_count UInt8,
        trip_distance Float32,
        fare_amount Float32,
        tip_amount Float32,
        tolls_amount Float32
    ) ENGINE = MergeTree()
    ORDER BY id;
""")

data = []
for i, row in df.iterrows():
    data.append([i, row["VendorID"], row["passenger_count"], row["trip_distance"], row["fare_amount"], row["tip_amount"], row["tolls_amount"]])

client.insert("nyc_taxi", data)

On voit ici qu'il y a un réel gain de temps lors de l'ajout par paquet.
L'ajout progressif des données est une limitation importante de ClickHouse et des bases de données orientées colonnes en général.

Vérifiez que les deux bases de données contiennent bien le même nombre de lignes.

In [13]:
result = sql_query("SELECT COUNT(*) FROM nyc_taxi;")
print("MySQL Total Rows:", result[0][0])

result = client.query("SELECT COUNT(*) FROM nyc_taxi;")
print("ClickHouse Total Rows:", result.result_rows[0][0])

MySQL Total Rows: 500000
ClickHouse Total Rows: 500000


## Comparaison 2 : Requêtes globales

On construit un jeu de requêtes pour étudier la différence dans le temps d'execution des requêtes selon la base de données. 

In [14]:
query_A = "SELECT * FROM nyc_taxi;"
query_B = "SELECT * FROM nyc_taxi ORDER BY passenger_count ASC, fare_amount DESC;"
query_C = "SELECT SUM(trip_distance) FROM nyc_taxi;"
query_D = "SELECT COUNT(*) FROM nyc_taxi;"
query_E = "SELECT COUNT(passenger_count) FROM nyc_taxi;"

In [15]:
iterations = 25
for query in [query_A, query_B, query_C, query_D, query_E] :
    t0 = time.time()
    for i in range(iterations) :
        resultsql = sql_query(query)
    t1 = time.time()
    for i in range(iterations) :
        resultcol = client.query(query)
    t2 = time.time()
    print("Requête : ", query)
    print("MySQL :      ", (t1 - t0)/iterations)
    print("ClickHouse : ", (t2 - t1)/iterations)
    print(" ")

Requête :  SELECT * FROM nyc_taxi;
MySQL :       0.7197241592407226
ClickHouse :  0.111017484664917
 
Requête :  SELECT * FROM nyc_taxi ORDER BY passenger_count ASC, fare_amount DESC;
MySQL :       0.8403416919708252
ClickHouse :  0.06148637771606445
 
Requête :  SELECT SUM(trip_distance) FROM nyc_taxi;
MySQL :       0.04455569267272949
ClickHouse :  0.005372638702392578
 
Requête :  SELECT COUNT(*) FROM nyc_taxi;
MySQL :       0.012353439331054688
ClickHouse :  0.0027822113037109376
 
Requête :  SELECT COUNT(passenger_count) FROM nyc_taxi;
MySQL :       0.12729467391967775
ClickHouse :  0.0035449981689453123
 


On constate que pour des requêtes globales, notamment les agrégations, ClickHous est plus performant que MySQL. En effet, le paradigme columnar est particulièrement efficace pour traiter des requêtes ne nécessitant qu'un faible nombre de colonnes et faisant appel à des agrégations.

À présent à vous d'essayer de faire une requête pour comparer les performances des deux paradigmes.

In [16]:
query = "SELECT * FROM nyc_taxi;"
iterations = 25
t0 = time.time()
for i in range(iterations) :
    resultsql = sql_query(query)
t1 = time.time()
for i in range(iterations) :
    resultcol = client.query(query)
t2 = time.time()
print("Requête : ", query)
print("MySQL :      ", (t1 - t0)/iterations)
print("ClickHouse : ", (t2 - t1)/iterations)

Requête :  SELECT * FROM nyc_taxi;
MySQL :       0.6609178733825684
ClickHouse :  0.0710689926147461


## Comparaison 3 : Requêtes spécifiques

On étudie à présent le cas où l'on souhaite accéder à une ligne spécifique.

In [17]:
query = "SELECT * FROM nyc_taxi WHERE id=1523"
iterations = 25
t0 = time.time()
for i in range(iterations) :
    resultsql = sql_query(query)
t1 = time.time()
for i in range(iterations) :
    resultcol = client.query(query)
t2 = time.time()
print("MySQL :      ", (t1 - t0)/iterations)
print("ClickHouse : ", (t2 - t1)/iterations)

MySQL :       0.012662525177001954
ClickHouse :  0.004894771575927734


In [18]:
query = "SELECT * FROM nyc_taxi WHERE id=30523 OR id=28645"
iterations = 25
t0 = time.time()
for i in range(iterations) :
    resultsql = sql_query(query)
t1 = time.time()
for i in range(iterations) :
    resultcol = client.query(query)
t2 = time.time()
print("MySQL :      ", (t1 - t0)/iterations)
print("ClickHouse : ", (t2 - t1)/iterations)

MySQL :       0.00038753509521484376
ClickHouse :  0.0027335453033447265


Les accès de lignes précises sont bien plus longues avec ClickHouse qu'avec MySQL !

In [19]:
query = "SELECT * FROM nyc_taxi WHERE id = 30523 OR tip_amount > 0"
iterations = 25
t0 = time.time()
for i in range(iterations) :
    resultsql = sql_query(query)
t1 = time.time()
for i in range(iterations) :
    resultcol = client.query(query)
t2 = time.time()
print("MySQL :      ", (t1 - t0)/iterations)
print("ClickHouse : ", (t2 - t1)/iterations)

MySQL :       0.443908634185791
ClickHouse :  0.05016204833984375


In [20]:
query = "SELECT * FROM nyc_taxi WHERE fare_amount > 1 AND fare_amount < 12 "
iterations = 25
t0 = time.time()
for i in range(iterations) :
    resultsql = sql_query(query)
t1 = time.time()
for i in range(iterations) :
    resultcol = client.query(query)
t2 = time.time()
print("MySQL :      ", (t1 - t0)/iterations)
print("ClickHouse : ", (t2 - t1)/iterations)

MySQL :       0.41467927932739257
ClickHouse :  0.05212672233581543


A l'inverse, le filtrage concernant une colonne est bien plus rapide avec ClickHouse.

In [21]:
query = "SELECT * FROM nyc_taxi WHERE fare_amount = 1.0"
iterations = 25
t0 = time.time()
for i in range(iterations) :
    resultsql = sql_query(query)
t1 = time.time()
for i in range(iterations) :
    resultcol = client.query(query)
t2 = time.time()
print("MySQL :      ", (t1 - t0)/iterations)
print("ClickHouse : ", (t2 - t1)/iterations)

MySQL :       0.07926018714904785
ClickHouse :  0.006500024795532227


## Comparaison 4 : Requête avec jointure

In [22]:
data = [(1, 0, 0.0),
        (2, 1, 1.0),
        (3, 2, 3.5),
        (5, 3, 6.0),
        (8, 4, 10.5)]

In [23]:
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS special_rule;")

cursor.execute("""
    CREATE TABLE special_rule (
        id INT PRIMARY KEY,
        passenger INT,
        reduction FLOAT
    );
""")

conn.commit()

for row in data:
    cursor.execute("""
        INSERT INTO special_rule (id, passenger, reduction)
        VALUES (%s, %s, %s)
    """, row)

conn.commit()
cursor.close()

client.command("DROP TABLE IF EXISTS special_rule;")

client.command("""
    CREATE TABLE special_rule (
        id UInt32,
        passenger UInt8,
        reduction Float32
    ) ENGINE = MergeTree()
    ORDER BY id;
""")

client.insert("special_rule", data)

In [24]:
resultsql = sql_query("SELECT * FROM special_rule")
print(resultsql)
resultcol = client.query("SELECT * FROM special_rule")
print(resultcol.result_rows)

[(1, 0, 0.0), (2, 1, 1.0), (3, 2, 3.5), (5, 3, 6.0), (8, 4, 10.5)]
[(1, 0, 0.0), (2, 1, 1.0), (3, 2, 3.5), (5, 3, 6.0), (8, 4, 10.5)]


In [25]:
query = "SELECT * FROM nyc_taxi JOIN special_rule ON nyc_taxi.passenger_count = special_rule.passenger WHERE tip_amount < 10 * fare_amount"
iterations = 25
t0 = time.time()
for i in range(iterations) :
    resultsql = sql_query(query)
t1 = time.time()
for i in range(iterations) :
    resultcol = client.query(query)
t2 = time.time()
print("MySQL :      ", (t1 - t0)/iterations)
print("ClickHouse : ", (t2 - t1)/iterations)

MySQL :       0.7587325668334961
ClickHouse :  0.11388928413391114


En conclusion, le columnar est plus pertinent pour les données analytiques ou historiques (informatique décisionnelle) qui ont besoin d'être lues rapidement et en grands volumes, qui sont peu modifiées ou agrégées. Ce paradigme est également recommandé pour les requêtes impliquant un nombre limité de colonnes ou un grand nombre de lignes. Enfin cela permet un requêtage plus performant, un meilleur taux de compression et de partition des données.

À l'inverse, pour des bases de données transactionnelles, avec de nombreux ajouts et mises à jour de données avec une grande variabilité et de multiples colonnes, le columnar est peu approprié et présente des performances décevantes voire rédhibitoires.

Pour tirer profit au maximum des atouts de chaque paradigme, en ligne ou en colonne, il est possible d'hybrider les technologies, comme c'est le cas sur les projets Snowflake ou Google Big Query.